In [1]:
import numpy as np
import scipy.special
import cvxpy as cp

from scipy.optimize import minimize

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile, Aer, IBMQ, execute
from qiskit.circuit import Parameter
from qiskit.tools.jupyter import *
from qiskit.visualization import *
#from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

from src import QAOA_utilities as QAOAut
from src import CellFreeQAOA as CFQAOA

# Loading your IBM Quantum account(s)
#provider = IBMQ.load_account()

In [2]:
M = 3 # number of cells
K = 3 # number of users
rho = 1 # max power
BW_max = 7 # max BW, M_hat in paper
rng = np.random.default_rng(2744)
#eta = np.random.rand(K) # power control fraction for each user
#alpha = np.random.rand(M,K)
#beta = np.random.rand(M,K)
eta = rng.random(K) # power control fraction for each user TO BE OTPIMIZED
alpha = rng.random((M,K))
beta = rng.random((M,K))
t = 1

In [3]:
def check_SINR(X, t, alpha, beta, rho, eta):
    snr = CFQAOA.SINR(X, eta, alpha, beta, rho )
    print(snr)
    return(snr >= t)

In [4]:
def check_soft_contraints(xvec, t, alpha, beta, rho, eta, lambda_soft):
    tg = 10.0 ** np.array([ -3, -2.8 , -2.5, -2.3 , -2.2, -2.0 , -1.8, -1.6 , -1.5, -1, -0.5]) # should find a good way to produce this
    #tg = np.array([0.01, 0.43, 0.4345440739052959, 0.44])
    Nt = len(tg)

    # Build auxiliary matrices
    Pt = np.concatenate((np.zeros((Nt,M*K)),np.eye(Nt)), axis = 1)
    Pk_list = []
    for k in range(K):
        Pk = np.concatenate((np.zeros((M,M*k)), np.eye(M), np.zeros((M,M*(K-k-1)+Nt))), axis = 1 )
        Pk_list.append(Pk)
    Pm_list = []
    for m in range(M):
        Pm = np.zeros((K,M*K+Nt))
        for k in range(K):
            Pm[k,m+M*k] = 1
        Pk_list.append(Pm)

    one_vec = np.ones(M*K+Nt)
    one_vec_tilde = np.concatenate((np.ones(M*K),np.zeros(Nt)))

    # build A and B

    A_list = []
    B_list = []

    b_tilde_list = []
    A_tilde_list = []

    for k in range(K):
        A_tilde = np.zeros((M,M))
        b_tilde = np.zeros(M)

        for m in range(M):
            for n in range(M):
                A_tilde[m,n] = rho*eta[k]*alpha[m,k]*alpha[n,k]
            b_tilde[m] = alpha[m,k]*(rho*(np.sum(np.multiply(eta,beta[m,:])) - beta[m,k]*eta[k]) + 1)

        A_tilde_list.append(A_tilde)
        b_tilde_list.append(b_tilde)

        A_list.append( Pk_list[k].T@A_tilde@Pk_list[k] )
        B_list.append( np.outer((Pt.T @ tg),(b_tilde.T @ Pk_list[k])) )

    C = np.sum( [ A_list[k] - B_list[k] for k in range(len(A_list)) ] , 0 )

    c_bar = np.sum(C,0)
    c_tilde = np.sum(C,1)
    
    
    obj = tg.T @ (Pt @ xvec) + lambda_soft* ( xvec.T @ (C @ xvec) )
    print('QAOA objective = ', obj)
    ccon = 0
    for k in range(K):
        print('A-B trace: ', np.trace(A_list[k] - B_list[k]))
        
        conk = xvec.T@((A_list[k] - B_list[k])@xvec)
        print(k,' A-B constraint: ', conk, '\n')
        ccon += conk
    print('Cumulative constrain (C) = ', ccon)
    return ccon

In [ ]:
tg = 10.0 ** np.array([ -3 , -2.5, -2.3 , -2.0, -1.5, -1, -0.5])
lambda_0 = 10
X_QAOA, t_QAOA, xt_QAOA = CFQAOA.optimize_links(BW_max, rho, eta, alpha, beta, t, lambda_0, tg)

----- Starting QAOA -----


In [ ]:
X_opt , t_opt = CFQAOA.bruteforce_SINR(M, K, BW_max, eta, alpha, beta, rho)

In [ ]:
print('QAOA X = \n', X_QAOA)
print('QAOA SNR = ',check_SINR(X_QAOA, t_QAOA, alpha, beta, rho, eta))
print('\n-------\n')
print('Optimal X = \n', X_opt)
print('Optimal SNR = ', check_SINR(X_opt, t_opt, alpha, beta, rho, eta))

In [ ]:
x_vec = X_QAOA.flatten('F')
x_QAOA = np.concatenate((x_vec,xt_QAOA))
check_soft_contraints(x_QAOA, t, alpha, beta, rho, eta, 10)